In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.optimizers import SGD, Adam
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D

from sklearn.model_selection import train_test_split, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import class_weight

from pdb import set_trace as t
import util


Using TensorFlow backend.


In [2]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [3]:
reload(util)

<module 'util' from 'util.pyc'>

In [4]:
vocab_size = 50000
max_review_length = 500
sentences, what_labels, _, _, _ = util.load_data(vocab_size, max_review_length)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(sentences, what_labels, test_size = 0.33)
class_rebalance = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [6]:
def create_model(arch):
    
    embedding_vector_length = 32
    dropout_rate = 0.2
    learn_rate = 0.005
    decay = 0
    
    if arch == 1:
        # with one ltsm
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
        model.add(Dropout(dropout_rate))
        model.add(LSTM(100))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(lr=learn_rate, decay=decay)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    elif arch ==2:
        # with 1D conv
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(LSTM(100))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(lr=learn_rate, decay=decay)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    elif arch == 3:
        # with one bidirectional
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
        model.add(Dropout(dropout_rate))
        model.add(Bidirectional(LSTM(100)))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(lr=learn_rate, decay=decay)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    elif arch == 4:
        # with two ltsm
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
        model.add(Dropout(dropout_rate))
        model.add(LSTM(100))
        model.add(Dropout(dropout_rate))
        model.add(LSTM(100))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(lr=learn_rate, decay=decay)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [7]:
model = KerasClassifier(build_fn=create_model, 
                        class_weight = class_rebalance,
                        epochs=1, batch_size=64, verbose = 1)

# define the grid search parameters
arch = [1,2,3]
param_grid = dict(arch=arch)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

/home/efs/nds.locus.io/berden/code/env/local/lib/python2.7/site-packages/sklearn/base.py:122: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/home/efs/nds.locus.io/berden/code/env/local/lib/python2.7/site-packages/sklearn/base.py:122: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/home/efs/nds.locus.io/berden/code/env/local/lib/python2.7/site-packages/sklearn/base.py:122: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/home/efs/nds.locus.io/berden/code/env/local/lib/

Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
18562/18562 [==============================] - 111s   


/home/efs/nds.locus.io/berden/code/env/local/lib/python2.7/site-packages/sklearn/base.py:122: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


Epoch 1/1
27842/27842 [==============================] - 185s - loss: 0.2650 - acc: 0.9074   


In [8]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.919079 using {'arch': 2}
0.907011 (0.005349) with: {'arch': 1}
0.919079 (0.005160) with: {'arch': 2}
0.918002 (0.007704) with: {'arch': 3}


# Running test notes